In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import onnx
from onnx2pytorch import ConvertModel
import torch

from patch_input_box import *
from run_verifier import *
import os

GRB_LICENSE_FILE="/root/ERAN/gurobi912/linux64/gurobi.lic"
os.environ["GRB_LICENSE_FILE"] = GRB_LICENSE_FILE
print(os.environ.get("GRB_LICENSE_FILE"))

model_path = "/root/ERAN/tf_verify/models/mnist_convSmallRELU__PGDK.onnx"
test_data_path = "/root/ERAN/data/mnist_test.csv"

images_dir = "/root/Projects/Nathan/Patch-Attack-Verification/images"

# Load the CSV
df = pd.read_csv(test_data_path, header=None)

# Extract labels and image pixels
labels = df.iloc[:, 0].values
pixels = df.iloc[:, 1:].values

/root/Projects/Nathan/Patch-Attack-Verification/util/ERAN/gurobi912/linux64/gurobi.lic
/root/ERAN/gurobi912/linux64/gurobi.lic


In [3]:
relaxed_example = [-0.42421292031695124, 1.6854916188347961, 1.6854916188347961, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203204175, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 0.073321440675146, -0.42421292031695124, 1.6854916188347961, -0.03698258211519052, -0.42421292031695124, -0.42421292031695124, -0.35079860957331993, -0.42421292031695124, -0.42421292031695124, -0.4242129203174435, -0.42421292031695124, -0.4242129203169283, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292019868423, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, -0.11610380081257617, -0.2045110968462241, 1.6854916188347961, -0.2212403866514669, -0.42421292031695124, -0.42421292031695124, -0.07638747128448876, -0.42421292031695124, -0.4242129203168071, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203158336, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129204155018, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, -0.42421292031695124, 1.6854916188347961, 1.6753014468329772, 1.6854916188347961, -0.42421292031695124, -0.1888533976283822, 1.6854916188347961, -0.42421292031695124, -0.42421292031651703, -0.4242129203171296, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203176984, -0.42421292032010255, -0.42421292030617247, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292016964884, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 0.042289538316776315, -0.42421292031695124, 0.23323666443667776, 0.007697163801411556, 1.6854916188347961, 1.6854916188347961, 1.3303240553297568, 1.6854916188347961, 1.6854916188347961, -0.42421292031695124, -0.424212920316715, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031842106, 0.05945993474070078, 2.808758138930695, 0.9631644377155191, -0.42421292032229185, -0.42421292031695124, -0.4242129205251693, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203077521, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.3556503267876238, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, -0.42421292031695124, -0.41510096733000984, -0.38507329865802403, 1.6854916188347961, -0.42421292031750857, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 0.6831433391038855, 2.783301675420514, 0.6195021803309917, -0.42421292034145336, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 0.3429431402270412, 1.6854916188347961, 1.2840983989403028, -0.42421292031695124, 1.5947394875693244, -0.42318458012200605, -0.42421292031695124, -0.42421292031688607, -0.4242129203175028, -0.42421292031695124, -0.4242129203183287, -0.42421292031695124, 1.2940985600766972, 2.6432911261146974, -0.42421292031695124, -0.42421292028141105, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031687197, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, -0.37445041349401736, 1.6854916188347961, -0.42421292031695124, -0.42421292031695124, -0.4242129203163393, -0.42421292031695124, -0.42421292031695124, 0.14855755702607853, 2.68147582137992, 1.48502203640294, -0.42421292031039554, -0.42421292030559754, -0.42421292031695124, -0.42421292031695124, -0.4242129204502714, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 0.6906434951116447, 1.5470914444966222, -0.42421292031695124, -0.4242129203283225, -0.42421292031695124, -0.42421292031695124, -0.4242129203252081, 0.6449586438394187, 2.8087581389305627, 0.3776657286174684, -0.42421292031588115, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 1.6854916188347961, 1.6854916188347961, 1.6854916188347961, 0.29379797200972996, 1.6854916188347961, 1.6854916188347961, 1.3590498208835025, -0.42421292031695124, -0.24745128455976997, 1.6854916188347961, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031777147, 2.1468900876665153, 2.4141829545233606, -0.2842023589199623, -0.42421292032470553, -0.4242129202311093, -0.4242129202301579, -0.42421292031695124, -0.4242129200996045, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203169551, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203169306, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203069927, -0.016909455790860055, 2.8087581389309144, 2.325085332237841, -0.4242129203173071, -0.42421292031695124, -0.42421292032841196, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031696706, -0.42421292031695124, -0.42421292031695124, -0.42421292031692664, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 0.7849691931444795, 2.8087581389306626, 2.0577924653812816, -0.42421292031695124, -0.42421292037721947, -0.42421292031695124, -0.42421292037906133, -0.42421292031695124, -0.4242129204480046, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203169465, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292035581354, -0.42421292029022645, -0.42421292031695124, -0.42421292031221247, -0.4242129203262329, -0.42421292031695124, 1.3577397188522304, 2.8087581389306697, 0.5558610215530956, -0.4242129203168256, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, 0.3012963380869696, 2.5923781990944, 2.1850747829320243, -0.32238705418481894, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129202944811, -0.42421292031695124, -0.42421292031334856, -0.42421292031695124, -0.42421292031695124, -0.42421292031647595, 1.1540879140415075, 2.821486370685737, 1.6759455127290086, -0.4242129203157869, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203196478, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292032069524, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203055013, -0.42421292031695124, -0.4242129203652667, -0.42421292031695124, -0.42421292034991837, -0.42421292031695124, 1.752314903259519, 2.8087581389306626, 0.6067739485734995, -0.42421292031751584, -0.42421292031695124, -0.4242129203232972, -0.42421292031695124, -0.42421292031695124, -0.4242129202887725, -0.42421292031695135, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129201981581, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292017071277, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203209533, -0.1187353340132391, 2.528737040319067, 2.3123571004827665, -0.424212920317005, -0.42421292031710195, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129204591756, -0.4242129204210041, -0.42421292028914664, -0.42421292031695124, -0.42421292031695124, -0.4242129201500526, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129202659458, 1.103174987021499, 2.8087581389306626, 1.5995761221986085, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292026628743, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203453287, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129208921781, 1.4977502682045205, 2.808758138928845, 1.3831961823597503, -0.4242129203094034, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292051857035, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129203767614, -0.42421292031695124, -0.42421292031695124, -0.4242129203890326, -0.42421292031695124, -0.42421292031695124, -0.42421292041987935, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292133892047, -0.42421292052923537, 2.4778241132925394, 2.808758138935369, 0.41585047224504085, -0.42421292038660396, -0.42421292034098346, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031386926, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292030621016, -0.42421292031695124, 0.35220926508562395, 2.7705734436565224, 2.808758138914458, 0.41585047233568057, -0.42421292081984685, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129202398492, -0.4242129203034104, -0.42421292077717115, 1.3704679506401798, 2.8087581389259624, 2.185074782947924, -0.38602822205931114, -0.42421291874316724, -0.4242129204694223, -0.42421292031695124, -0.42421292031695124, -0.42421292023478535, -0.42421292031695124, -0.42421292031695124, -0.4242129202123617, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421291984526077, -0.2969305906750366, 2.3123571004719286, 2.808758138898522, 1.1159032188257885, -0.424212920075421, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129202231889, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292032442065, -0.42421292031695124, -0.42421292032071417, -0.42421292031695124, -0.3605717554959939, 2.095977160648142, 1.815956062129594, -0.296930591434056, -0.42421292093927787, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129201294628, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292062490157, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129206358396, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129206118793, -0.4242129207158373, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421291958578133, -0.42421292031695124, -0.42421292028839785, -0.4242129203621775, -0.42421292031695124, -0.42421292031695124, -0.4242129196333067, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.4242129207127953, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124, -0.42421292031695124]

In [4]:
import torch
import onnx
import numpy as np
from onnx2pytorch import ConvertModel

def get_pixel_influence(model_source, input_image, correct_class_idx, target_class_idx):
    """
    Computes the gradient of (Z_correct - Z_target) w.r.t the input pixels.
    Automatically reshapes flat inputs (784) to (1, 1, 28, 28).
    """
    
    # 1. Handle Model Loading
    if isinstance(model_source, str):
        onnx_model = onnx.load(model_source)
        pytorch_model = ConvertModel(onnx_model)
        pytorch_model.eval()
    else:
        pytorch_model = model_source

    # 2. Prepare Input Tensor & Reshape
    # Convert numpy to torch if needed
    if not torch.is_tensor(input_image):
        input_tensor = torch.from_numpy(input_image).float()
    else:
        input_tensor = input_image.clone().float()

    # --- RESHAPING LOGIC ---
    # Case A: Input is flat vector (784) or (1, 784)
    if input_tensor.numel() == 784:
        input_tensor = input_tensor.view(1, 1, 28, 28)
    # Case B: Input is (28, 28) -> Add batch and channel dims
    elif input_tensor.shape == (28, 28):
        input_tensor = input_tensor.unsqueeze(0).unsqueeze(0)
    # Case C: Input is (1, 28, 28) -> Add batch dim
    elif input_tensor.shape == (1, 28, 28):
        input_tensor = input_tensor.unsqueeze(0)
    
    # Check shape is correct now
    assert input_tensor.shape == (1, 1, 28, 28), f"Input shape mismatch: {input_tensor.shape}"

    # 3. Enable Gradients
    input_tensor.requires_grad = True

    # 4. Forward Pass
    outputs = pytorch_model(input_tensor)
    
    # 5. Compute Margin & Backward
    # outputs is [1, 10]
    z_correct = outputs[0, correct_class_idx]
    z_target  = outputs[0, target_class_idx]
    
    margin = z_correct - z_target

    if pytorch_model.parameters():
        pytorch_model.zero_grad()
        
    margin.backward()

    # 6. Get Gradients
    grads = input_tensor.grad.data.abs()
    
    return grads

# --- Helper to get top K indices (compatible with Gurobi x0...x783) ---
def get_top_k_pixels(influence_map, k=30):
    # Flatten strictly to match Gurobi's variable ordering (x0, x1, ...)
    flat_influence = influence_map.view(-1)
    
    top_k_values, top_k_indices = torch.topk(flat_influence, k)
    
    return top_k_indices.tolist()

In [5]:
grads = get_pixel_influence(
    model_source=model_path,  # Pass the converted model to save time!
    input_image=np.array(relaxed_example).reshape(1, 1, 28, 28),  # Example input
    correct_class_idx=3,         # e.g., 'Cat'
    target_class_idx=5           # e.g., 'Dog' (the class the solver is worried about)
)

# Get the split candidates
pixels_to_split = get_top_k_pixels(grads, k=30)

In [6]:
print("Top pixels to consider for splitting:", pixels_to_split)

Top pixels to consider for splitting: [233, 128, 232, 205, 268, 230, 231, 433, 571, 405, 229, 572, 156, 241, 296, 158, 544, 130, 546, 516, 228, 203, 406, 202, 227, 201, 200, 261, 574, 518]


In [8]:
print(len(grads.view(-1)))  # Should be 784

784


In [9]:
def get_top_k_pixels_in_patch(influence_map, k=30, patch_x=0, patch_y=0, patch_size=10, img_w=28, img_h=28):
    """
    Finds the top K influential pixels RESTRICTED to a specific patch area.
    Returns the GLOBAL flat indices (0-783) compatible with the solver.
    
    Args:
        influence_map: Tensor of gradients (shape can be flat or formatted).
        patch_x: Top-left column index.
        patch_y: Top-left row index.
        patch_size: Width/Height of the square patch.
    """
    # 1. Ensure map is 2D (H, W) so we can slice it easily
    grad_2d = influence_map.view(img_h, img_w)
    
    # 2. Collect candidates strictly from the patch area
    candidates = []
    
    # Define bounds (handling edge cases where patch might go off-screen)
    y_start = max(0, patch_y)
    y_end   = min(img_h, patch_y + patch_size)
    x_start = max(0, patch_x)
    x_end   = min(img_w, patch_x + patch_size)

    # 3. Iterate ONLY through the patch pixels
    for r in range(y_start, y_end):      # Rows (y)
        for c in range(x_start, x_end):  # Cols (x)
            
            # The gradient magnitude for this pixel
            val = grad_2d[r, c].item()
            
            # The GLOBAL index (0 to 783)
            # Formula: row_index * width + col_index
            global_idx = r * img_w + c
            
            candidates.append((val, global_idx))

    # 4. Sort by gradient value (descending)
    candidates.sort(key=lambda x: x[0], reverse=True)
    
    # 5. Take top K (or all if patch is smaller than K)
    top_k = candidates[:k]
    
    # 6. Return only the indices
    return [x[1] for x in top_k]

# --- USAGE ---

# 1. Compute gradients as before
grads = get_pixel_influence(
    model_source=model_path,
    input_image=np.array(relaxed_example).reshape(1, 1, 28, 28),
    correct_class_idx=3,
    target_class_idx=5
)

# 2. Get the restricted indices
# x=0, y=0 (Top-left corner), size=10
pixels_to_split = get_top_k_pixels_in_patch(
    grads, 
    k=30, 
    patch_x=0, 
    patch_y=0, 
    patch_size=10
)

print("Top pixels inside patch (0,0):", pixels_to_split)

Top pixels inside patch (0,0): [233, 232, 205, 230, 231, 229, 228, 203, 202, 227, 201, 200, 261, 199, 226, 198, 257, 258, 256, 149, 225, 255, 120, 145, 197, 254, 121, 170, 144, 171]
